# Training A ball tracking Model 

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import glob, os


In [12]:
if torch.backends.mps.is_available():
    device = "mps"
    print("🔥 Using Apple MPS GPU")
elif torch.cuda.is_available():
    device = "cuda"
    print("🚀 Using NVIDIA CUDA GPU")
else:
    device = "cpu"
    print("⚙️ Using CPU")

device


🔥 Using Apple MPS GPU


'mps'

In [17]:
model = YOLO("yolo11s.pt")

In [18]:
caches = glob.glob("data/datasets/baseball/**/labels.cache", recursive=True)

print("Found caches:", caches)
for c in caches:
    os.remove(c)
    print("Deleted:", c)

Found caches: ['data/datasets/baseball/valid/labels.cache', 'data/datasets/baseball/train/labels.cache']
Deleted: data/datasets/baseball/valid/labels.cache
Deleted: data/datasets/baseball/train/labels.cache


In [19]:
results = model.train(
    data="data/baseball.yaml",  # this is the YAML file
    epochs=30,
    imgsz=640,
    batch=16,
    device="mps",
)

Ultralytics 8.3.228 🚀 Python-3.10.19 torch-2.9.1 MPS (Apple M2 Max)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/baseball.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train30, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pr

Corrupt JPEG data: 963 extraneous bytes before marker 0xd9


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.8s/it 21.6s0.9s
                   all        375        377      0.804      0.698       0.76      0.311

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/30      9.46G      2.225      1.467     0.8656         10        640: 100% ━━━━━━━━━━━━ 284/284 1.3it/s 3:35<0.7s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 8% ━─────────── 1/12 2.2s/it 0.6s<23.7s

Corrupt JPEG data: 963 extraneous bytes before marker 0xd9


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.6it/s 7.4s0.6s
                   all        375        377      0.717      0.603      0.656      0.236

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/30      9.47G       2.22      1.332     0.8581         12        640: 100% ━━━━━━━━━━━━ 284/284 1.4it/s 3:30<0.7s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 8% ━─────────── 1/12 4.7s/it 1.4s<51.3s

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.6it/s 7.4s0.6s
                   all        375        377      0.931      0.897      0.943      0.492

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      30/30      8.46G      1.608     0.7089     0.8334          6        640: 100% ━━━━━━━━━━━━ 284/284 1.2it/s 3:51<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 8% ━─────────── 1/12 2.3s/it 0.7s<25.3s

Corrupt JPEG data: 963 extraneous bytes before marker 0xd9


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.6it/s 7.5s0.6s
                   all        375        377      0.919      0.897      0.942      0.494

30 epochs completed in 1.847 hours.
Optimizer stripped from /opt/homebrew/runs/detect/train30/weights/last.pt, 19.2MB
Optimizer stripped from /opt/homebrew/runs/detect/train30/weights/best.pt, 19.2MB

Validating /opt/homebrew/runs/detect/train30/weights/best.pt...
Ultralytics 8.3.228 🚀 Python-3.10.19 torch-2.9.1 MPS (Apple M2 Max)
YOLO11s summary (fused): 100 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 8% ━─────────── 1/12 7.9s/it 2.4s<1:26

Corrupt JPEG data: 963 extraneous bytes before marker 0xd9


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.1it/s 10.5s.7s
                   all        375        377      0.918      0.897      0.942      0.494
              baseball        375        377      0.918      0.897      0.942      0.494
Speed: 0.1ms preprocess, 6.5ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to /opt/homebrew/runs/detect/train30


In [23]:


# Load your trained model
model = YOLO("/opt/homebrew/runs/detect/train30/weights/best.pt")

# Optional: force MPS (if needed)
# model.to("mps")

# Process a baseball video
video_path = "data/baseball_pitch.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {video_path}")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
output_path = "tracked_pitch.mp4"

# Create output video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

ball_trajectory = []
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO11 on the frame
    results = model(frame, conf=0.35, iou=0.45, verbose=False)[0]

    # Boxes, classes, confidences
    boxes = results.boxes.xyxy.cpu().numpy() if results.boxes.xyxy is not None else []
    classes = results.boxes.cls.cpu().numpy() if results.boxes.cls is not None else []
    scores = results.boxes.conf.cpu().numpy() if results.boxes.conf is not None else []

    for box, cls, score in zip(boxes, classes, scores):
        class_name = model.names[int(cls)].lower()

        # Adjust this depending on what you named the class in your YAML
        if class_name in ["ball", "baseball", "sports ball"]:
            x1, y1, x2, y2 = map(int, box)
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            # Add to trajectory
            ball_trajectory.append((frame_idx, center_x, center_y))

            # Draw box and center
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

    # Draw trajectory (connect recent points)
    if len(ball_trajectory) > 1:
        for i in range(1, len(ball_trajectory)):
            if ball_trajectory[i][0] - ball_trajectory[i - 1][0] <= 3:
                pt1 = (ball_trajectory[i - 1][1], ball_trajectory[i - 1][2])
                pt2 = (ball_trajectory[i][1], ball_trajectory[i][2])
                cv2.line(frame, pt1, pt2, (255, 0, 0), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"Saved tracked video to: {output_path}")


Saved tracked video to: tracked_pitch.mp4
